In [1]:
#libraries and globals
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
torch.manual_seed(123)


In [2]:
#define the neural network (Listing A4)
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            
            #1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            #2nd hiddenlayer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            #output layer
            torch.nn.Linear(20,num_outputs),
        )

    def forward(self,x):
        logits = self.layers(x)
        return logits


In [3]:
#Create a dataset (Listing A5)
def StoreData():
    X_train = torch.tensor([
        [-1.2, 3.1],
        [-0.9, 2.9],
        [-0.5, 2.6],
        [2.3, -1.1],
        [2.7, -1.5]
   ])
    y_train = torch.tensor([0,0,0,1,1])

    X_test = torch.tensor([
        [-0.8, 2.8],
        [2.6,-1.6],
    ])
    y_test = torch.tensor([0,1])

    return(X_train, y_train,X_test,y_test)

In [4]:
#Define a custom dataset class (Listing A6)
class ToyDataset(Dataset):
    def __init__(self,X,y):
        self.features = X
        self.labels = y

    def __getitem__(self,index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]


In [5]:
#Initialize Data Loaders (Listing A7)

def LoadData(train_ds,test_ds):
    train_loader = DataLoader(
        dataset=train_ds,
        batch_size=2,  #controls the grouping, change to 2 and compare output.
        shuffle=True, #controls shuffling of X_train, change to 2 and compare output
        num_workers=0,
        drop_last=True
    )
    test_loader=DataLoader(
        dataset=test_ds,
        batch_size=1,
        shuffle=False,
        num_workers=0
    )

    return(train_loader, test_loader)


In [6]:
#Training step (Listing A7)

def TrainData(train_loader,inp,out):
    
    model = NeuralNetwork(num_inputs=inp, num_outputs=out) 
    torch.manual_seed(123)
    optimizer = torch.optim.SGD(
        model.parameters(), lr=0.5
    ) 
    num_epochs = 3
    for epoch in range(num_epochs): 
        model.train()
        for batch_idx, (features,labels) in enumerate (train_loader):
            logits = model(features)   #forward pass
            loss = F.cross_entropy(logits,labels) #compute loss
            optimizer.zero_grad()  #clear gradients leftover from the last pass
            loss.backward()  #compute gradients
            optimizer.step() #update weights
            print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}" 
                f" | Batch {batch_idx:03d}/{len(train_loader):03d}" 
                f" | Train Loss: {loss:.2f}")
            print()
    return model
   

In [7]:
def MakePredictions(model):
    model.eval()
    with torch.no_grad():
        outputs = model(X_train)
    print(outputs)
    print()

    torch.set_printoptions(sci_mode=False)
    probas = torch.softmax(outputs,dim=1)
    #print(probas)
    predictions = torch.argmax(probas, dim=1)
    print(predictions)

    #cab apply argmax directly to outputs
    predictions = torch.argmax(outputs,dim=1)
    print(predictions)

In [8]:
def compute_accuracy(model,dataloader):
    model = model.eval()
    correct = 0.0
    total_examples = 0
    for idx, (features, labels) in enumerate (dataloader):
        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)
    return (correct/total_examples).item()

In [9]:
#Invoke above functions

X_train, y_train,X_test,y_test = StoreData()
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

#print(len(train_ds))

train_loader, test_loader = LoadData(train_ds,test_ds)
#for idx, (x,y) in enumerate(train_loader):
   #print(f"Batch {idx+1}:", x,y)

model = TrainData(train_loader,2,2)

MakePredictions(model)
print(compute_accuracy(model,train_loader))

torch.save(model.state_dict(), "model.pth")




Epoch: 001/003 | Batch 000/002 | Train Loss: 0.67

Epoch: 001/003 | Batch 001/002 | Train Loss: 0.24

Epoch: 002/003 | Batch 000/002 | Train Loss: 0.01

Epoch: 002/003 | Batch 001/002 | Train Loss: 0.34

Epoch: 003/003 | Batch 000/002 | Train Loss: 0.10

Epoch: 003/003 | Batch 001/002 | Train Loss: 0.04



In [26]:
#Load a previously trained model
#This generates a warning not noted on p. 279 im Raschka
model = NeuralNetwork(2,2)
model.load_state_dict(torch.load("model.pth"))
MakePredictions(model)
print(compute_accuracy(model,train_loader))


tensor([[ 2.3852, -3.5150],
        [ 2.1389, -3.1915],
        [ 1.7915, -2.7423],
        [-1.4977,  1.4431],
        [-1.7264,  1.6689]])

tensor([0, 0, 0, 1, 1])
tensor([0, 0, 0, 1, 1])
1.0


/var/folders/4y/skknyhs53qd2zdrq2jyjnx800000gp/T/ipykernel_12037/1584616285.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth")